# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Te Anau,-45.4167,167.7167,69.10,54,100,5.91,NZ,1612754873
1,1,Cape Town,-33.9258,18.4232,66.99,72,0,14.97,ZA,1612754841
2,2,Narsaq,60.9167,-46.0500,39.20,28,20,39.12,GL,1612754878
3,3,Hobart,-42.8794,147.3294,66.20,45,40,18.41,AU,1612754747
4,4,Petropavlovsk-Kamchatskiy,53.0452,158.6483,24.80,80,90,8.95,RU,1612754863


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [10]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

humidity = cities_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [11]:
# Criteria: Temp between 65 - 75, humidity < 50%, winds < 17 mph, cloudiness <= 40%

nice_cities = cities_df.loc[(cities_df['Max Temp'] <= 75) & (cities_df['Max Temp'] >= 65) 
               & (cities_df['Humidity'] < 50) & (cities_df['Wind Speed'] < 17) & (cities_df['Cloudiness'] <= 40)]
nice_cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,Mangan,27.5167,88.5333,68.05,21,0,3.83,IN,1612754905
141,141,Nago,26.5881,127.9761,71.60,46,1,12.66,JP,1612755120
235,235,Māngrol,21.1167,70.1167,73.81,28,0,9.55,IN,1612755287
239,239,Burnie,-41.0667,145.9167,71.01,40,17,3.74,AU,1612755298
274,274,Along,28.1667,94.7667,69.26,26,0,0.63,IN,1612755358
320,320,Nishihara,26.1842,127.7558,71.60,37,20,14.97,JP,1612755438
393,393,Makkah al Mukarramah,21.4267,39.8261,68.32,25,0,15.26,SA,1612755571
428,428,Atoyac,20.0167,-103.5333,68.00,28,1,8.05,MX,1612755632
448,448,Cooma,-36.2333,149.1333,71.60,49,17,11.50,AU,1612755671
490,490,Pālanpur,24.1667,72.4333,74.23,20,0,5.99,IN,1612755750


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = nice_cities
hotel_df["Hotel Name"] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add keyword to params dict
    params['location'] = str(hotel_df.loc[index,"Lat"]) + "," + str(hotel_df.loc[index,"Lng"])

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    print(results)
    hotel_df.loc[index,"Hotel Name"] = results[0]['name']

C:\Users\Kelly\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[{'business_status': 'CLOSED_TEMPORARILY', 'geometry': {'location': {'lat': 27.500896, 'lng': 88.534784}, 'viewport': {'northeast': {'lat': 27.5022305802915, 'lng': 88.5361652802915}, 'southwest': {'lat': 27.49953261970849, 'lng': 88.53346731970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Mout View', 'permanently_closed': True, 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102381445812564757615">Noorsang Lama</a>'], 'photo_reference': 'ATtYBwKhnK7QnPGKZb3i1egZZRow5mX3vw6ydlQX-SsaOXoLTybgobEDh60WE3XqVHhpmW3btI8j2athcVXQmrLlcMB2PyGky9wG2kiSNPxRL7pfCRvgQPZEfOj0D_IVOKjD9tt4nfR1-5dfL3XuZaKpfrjIPU_6ZPpwP-9opo_ua38UUn1-', 'width': 3000}], 'place_id': 'ChIJk6Nf1Ri_5jkR0DWNEWUzrsM', 'plus_code': {'compound_code': 'GG2M+9W Mangan, Sikkim, India', 'global_code': '7MVCGG2M+9W'}, 'rating': 3, 'reference': 'ChIJk6Nf1Ri_5jkR0DWNEWUzrsM', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_

C:\Users\Kelly\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 26.5934393, 'lng': 127.96508}, 'viewport': {'northeast': {'lat': 26.5945919802915, 'lng': 127.9664311302915}, 'southwest': {'lat': 26.5918940197085, 'lng': 127.9637331697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Yugaf Inn Okinawa', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104092286118228798083">浦本洋伸</a>'], 'photo_reference': 'ATtYBwIt2M4onolnDbCfimlsP-bvkJOeX6CE8vS83Yu9NGITPxJdP2VLZGAtqlJn5kN8OsaIY9IMd5DIzVlBgHOA04b70em5ZVH8JXkLk4_pZWu53Lax5RzfF_uokTu-29PtC-tDp_6jOGr7ey9ApUT6LfwfzX2gKQAmkWgBJKJ3CQY0az5t', 'width': 4608}], 'place_id': 'ChIJIcdjU27_5DQRgCCZxgvYCls', 'plus_code': {'compound_code': 'HXV8+92 Nago, Okinawa, Japan', 'global_code': '7QR9HXV8+92'}, 'rating': 4.1, 'reference': 'ChIJIcdjU27_5DQRgCCZxgvYCls', 'scope': 'GOOGLE', 'types': ['lodging', 'restau

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 21.1433136, 'lng': 70.08869609999999}, 'viewport': {'northeast': {'lat': 21.1447922802915, 'lng': 70.0898874302915}, 'southwest': {'lat': 21.1420943197085, 'lng': 70.08718946970848}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel The Royal Honours', 'photos': [{'height': 3464, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105904380904384570662">Mukul Jani</a>'], 'photo_reference': 'ATtYBwLf5aj5AJkiECubCqvrVbwTHG1P7A2ylDFk4hHJX--MZGNRui3cjJB6MX-MKxJ664qlzyPLSVSj9W6GWfh9ePaX_jQiZTE-aKhVW6ctFY3OJ3rJU5M7EyzyfpvR1dRwCmF2kI6CqJwBQnWRLCEKI3QW1ZcsHES45FilAEg7ViJNFO3b', 'width': 4618}], 'place_id': 'ChIJ3TvFM0hC_TsRZKe1dkYjSA8', 'plus_code': {'compound_code': '43VQ+8F Mangrol, Gujarat, India', 'global_code': '7JHG43VQ+8F'}, 'rating': 3.9, 'reference': 'ChIJ3TvFM0hC_TsRZKe1dkYjSA8', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establ

...

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 28.168064, 'lng': 94.80035199999999}, 'viewport': {'northeast': {'lat': 28.16947098029151, 'lng': 94.8017104302915}, 'southwest': {'lat': 28.1667730197085, 'lng': 94.79901246970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel West', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103225268643741136951">Dan Shalev</a>'], 'photo_reference': 'ATtYBwIky_X6kzwsRx-xl_nfI0ERY9_hgLjn9mpN_F9VYTB5zlflUI2_9UxT-BfLQJr8yRlLKsOSwtrwZIJn-nXAXBqDv1yXCeYQ4lhY-OHkkmDutA2TIaKE7TTTGaVFCmD1n3PrusNkz2zfLY7NihloWYelQH37p208VEoE-WdvbDxjRoMk', 'width': 4032}], 'place_id': 'ChIJgU-dsoJrQDcRCkBpibDMmno', 'plus_code': {'compound_code': '5R92+64 Aalo, Arunachal Pradesh', 'global_code': '7MWP5R92+64'}, 'rating': 3.5, 'reference': 'ChIJgU-dsoJrQDcRCkBpibDMmno', 'scope': 'GOOGLE', 'types': ['lodging', 'poi

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 26.2029209, 'lng': 127.7600632}, 'viewport': {'northeast': {'lat': 26.20425563029151, 'lng': 127.7614440302915}, 'southwest': {'lat': 26.2015576697085, 'lng': 127.7587460697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Minshuku Agaihama', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3468, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107437102851478364346">chong chiu</a>'], 'photo_reference': 'ATtYBwLwQgcbxTxIMzlhMDvPXk6tr8XCWBmoptogq8deOEbZlumo922HDUtO5biKvXNiO6smjYZKXeObxN7qD-b6rLUD7bUom4LntlqmEfMED2iCDJUirouWrSVhOB25MTYFj4Blr3wult17j82vlhsDv2WiFMSX1dp7EzYmVY49g1C-8hfm', 'width': 4624}], 'place_id': 'ChIJl6yONdNt5TQRLRHuoeM1GfY', 'plus_code': {'compound_code': '6Q36+52 Yonabaru, Okinawa, Japan', 'global_code': '7QR96Q36+52'}, 'rating': 4.1, 'reference': 'ChIJl6yONdNt5TQRLRHuoeM1GfY', 'scope': 'GOOGLE', 'types': ['lodging', 

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 21.418034, 'lng': 39.82556899999999}, 'viewport': {'northeast': {'lat': 21.4196736802915, 'lng': 39.82804040000001}, 'southwest': {'lat': 21.4169757197085, 'lng': 39.8247452}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Makkah Clock Royal Tower, A Fairmont Hotel', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2736, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117571296166179502908">عبدالله عبيد</a>'], 'photo_reference': 'ATtYBwLPCVbPXIXXTR0f81IDv3rUYRul10N4YAMoTv0Q_DONitAJExwAG4Jg-Sax2zZBQJ9CJ5OHvj13WUWVbN2t7hMmwk00Em6bxEnUn5A2MhhcC4N6lZOelwZzq35D5ng1jYRcgywdbriCBMeyq_3i-4YMOFrvAcymwzi6VvXrL9shvWdM', 'width': 3648}], 'place_id': 'ChIJN1Hy6rcEwhURanDxKxvnCIM', 'plus_code': {'compound_code': 'CR9G+66 Mecca Saudi Arabia', 'global_code': '7GHXCR9G+66'}, 'rating': 4.6, 'reference': 'ChIJN1Hy6rcEwhURanDxKxvnCIM', 'scope': 'GOOGLE', 'ty

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 20.0096594, 'lng': -103.5171145}, 'viewport': {'northeast': {'lat': 20.0109706302915, 'lng': -103.5157732697085}, 'southwest': {'lat': 20.0082726697085, 'lng': -103.5184712302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'OYO Hotel Del Portal', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2322, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114574764913278020794">luna lunita</a>'], 'photo_reference': 'ATtYBwIg45Mef-8Kkiy8JSXAkzZG2tWtFzeuWsLpKZi0FvmVLJKPuxVOWzBCPArh41Oed0JiqfOBr4bp1dFQmXLqs8dqx0quUBHeDcZGPj6Vi0TVKJPQGWcAt4Z5z6ponxnAGwt5LqN9wugx9Kii5MFb99CbCo8KEbDm5Iw0EooKrykkr5vt', 'width': 4128}], 'place_id': 'ChIJl_a6j8hiL4QR7A_2oG7LIk4', 'plus_code': {'compound_code': '2F5M+V5 Atoyac, Jalisco, Mexico', 'global_code': '75GR2F5M+V5'}, 'rating': 4.2, 'reference': 'ChIJl_a6j8hiL4QR7A_2oG7LIk4', 'scope': 'GOOGLE', 'types': ['lodgi

[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 24.1739162, 'lng': 72.4164139}, 'viewport': {'northeast': {'lat': 24.1752651802915, 'lng': 72.4177628802915}, 'southwest': {'lat': 24.17256721970849, 'lng': 72.41506491970848}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Lajwanti', 'photos': [{'height': 1920, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113708415880439422266">Hotel Lajwanti</a>'], 'photo_reference': 'ATtYBwL7UppdITA-s6gn9Y_K0L-BYL7EqM6EfWwJBZ4v98fpFv3nfK5njt5oipdHE2881byrppO_7bSt4V9ZsxkYZECrJ7Z0EjSho1y8YXbLdj0VMmUxbljaH1wFsk15M24lFjK6lxd14ySgMIw0abegpSO9QOWE5KyXWbDDPt4486RT5VUk', 'width': 1080}], 'place_id': 'ChIJiTGoOkPpXDkR7K6LQ9j68FU', 'plus_code': {'compound_code': '5CF8+HH Palanpur, Gujarat, India', 'global_code': '7JPJ5CF8+HH'}, 'rating': 3.8, 'reference': 'ChIJiTGoOkPpXDkR7K6LQ9j68FU', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'],

In [13]:
#for hotel in hotel_df["Hotel Name"]:
#    if hotel_df.loc[hotel,"Hotel Name"].len < 1:
#        hotel_df[hotel, "Hotel Name"] = "No Hotel Found"
[hotel_df.loc[hotel,"Hotel Name"] = "n/a" for hotel in hotel_df["Hotel Name"] if hotel_df.loc[hotel,"Hotel Name"] == '']

SyntaxError: invalid syntax (<ipython-input-13-b92d44fe72a1>, line 4)

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locationss = hotel_df[["Lat", "Lng"]]
locationss.values

array([[  27.5167,   88.5333],
       [  26.5881,  127.9761],
       [  21.1167,   70.1167],
       [ -41.0667,  145.9167],
       [  28.1667,   94.7667],
       [  26.1842,  127.7558],
       [  21.4267,   39.8261],
       [  20.0167, -103.5333],
       [ -36.2333,  149.1333],
       [  24.1667,   72.4333]])

In [15]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locationss.values, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))